In [1]:
# SG by category model 
using Pkg, JuMP, Ipopt, DataFrames, CSV, Random, Distributions

In [5]:
#pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_plyr_weights.csv") # nodsr weights
#hweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_hole_weights.csv")


pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_byhole_sg_plyr_weights.csv") # dsr weights
hweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_byhole_sg_hole_weights.csv")

real_data = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_2018_data.csv") # scaled data

real_data[!, :SG_total] = real_data[!, :SG_appr] + real_data[!, :SG_short]  +real_data[!, :SG_drive] + real_data[!, :SG_putt] 
n_real = length(real_data[!,:player])
num_plyrs = length(unique(real_data[!,:player]))
num_holes = length(unique(real_data[!,:course_hole]))

666

# NO DSR 

In [4]:
mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_drive[1:num_plyrs])
@variable(mod_real, p_sg_appr[1:num_plyrs])
@variable(mod_real, p_sg_short[1:num_plyrs])
@variable(mod_real, p_sg_putt[1:num_plyrs])

@variable(mod_real, h_sg_drive[1:num_holes])
@variable(mod_real, h_sg_appr[1:num_holes])
@variable(mod_real, h_sg_short[1:num_holes])
@variable(mod_real, h_sg_putt[1:num_holes])

p_sg_drive_ = [p_sg_drive[i+1] for i in real_data[!,:p_idx]]
p_sg_appr_ = [p_sg_appr[i+1] for i in real_data[!,:p_idx]]
p_sg_short_ = [p_sg_short[i+1] for i in real_data[!,:p_idx]]
p_sg_putt_ = [p_sg_putt[i+1] for i in real_data[!,:p_idx]]

h_sg_drive_ = [h_sg_drive[i+1] for i in real_data[!,:h_idx]]
h_sg_appr_ = [h_sg_appr[i+1] for i in real_data[!,:h_idx]]
h_sg_short_ = [h_sg_short[i+1] for i in real_data[!,:h_idx]]
h_sg_putt_ = [h_sg_putt[i+1] for i in real_data[!,:h_idx]]

hsg_wmean_drive = @expression(mod_real, sum(h_sg_drive[i]*(hweight_df[!,:weight][i]) for i in 1:num_holes))
hsg_wmean_appr = @expression(mod_real, sum(h_sg_appr[i]*(hweight_df[!,:weight][i]) for i in 1:num_holes))
hsg_wmean_short = @expression(mod_real, sum(h_sg_short[i]*(hweight_df[!,:weight][i]) for i in 1:num_holes))
hsg_wmean_putt = @expression(mod_real, sum(h_sg_putt[i]*(hweight_df[!,:weight][i]) for i in 1:num_holes))

hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 
hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 
hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 
hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0)

psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))

psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 
psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 
psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 
psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0)

sg_drive_constr = @constraint(mod_real, [i=1:num_holes], -5 <= h_sg_drive[i] <= 5) 
sg_appr_constr = @constraint(mod_real, [i=1:num_holes],  -5 <= h_sg_appr[i] <= 5) 
sg_short_constr = @constraint(mod_real, [i=1:num_holes],  -5 <= h_sg_short[i] <= 5) 
sg_putt_constr = @constraint(mod_real, [i=1:num_holes],  -5 <= h_sg_putt[i] <= 5) 

psg_drive_constr = @constraint(mod_real, [i=1:num_plyrs], -5 <= p_sg_drive[i] <= 5) 
psg_appr_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_appr[i] <= 5) 
psg_short_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_short[i] <= 5) 
psg_putt_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_putt[i] <= 5) 


@NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i] - 2*p_sg_drive_[i]*h_sg_drive_[i] + h_sg_drive_[i]*h_sg_drive_[i] - 2*p_sg_drive_[i]*real_data[!,:SG_drive][i] + 2*h_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i] - 2*p_sg_appr_[i]*h_sg_appr_[i] + h_sg_appr_[i]*h_sg_appr_[i] - 2*p_sg_appr_[i]*real_data[!,:SG_appr][i] + 2*h_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i] - 2*p_sg_short_[i]*h_sg_short_[i] + h_sg_short_[i]*h_sg_short_[i] - 2*p_sg_short_[i]*real_data[!,:SG_short][i] + 2*h_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i] - 2*p_sg_putt_[i]*h_sg_putt_[i] + h_sg_putt_[i]*h_sg_putt_[i] - 2*p_sg_putt_[i]*real_data[!,:SG_putt][i] + 2*h_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i]) for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

h_idx_df = DataFrame(idx = unique(real_data[!,:h_idx]), course_hole = unique(real_data[!,:course_hole]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_d_label = [value.(p_sg_drive)[i+1] for i in p_idx_df[!,:idx]]
p_a_label = [value.(p_sg_appr)[i+1] for i in p_idx_df[!,:idx]]
p_s_label = [value.(p_sg_short)[i+1] for i in p_idx_df[!,:idx]]
p_p_label = [value.(p_sg_putt)[i+1] for i in p_idx_df[!,:idx]]

h_d_label = [value.(h_sg_drive)[i+1] for i in h_idx_df[!,:idx]]
h_a_label = [value.(h_sg_appr)[i+1] for i in h_idx_df[!,:idx]]
h_s_label = [value.(h_sg_short)[i+1] for i in h_idx_df[!,:idx]]
h_p_label = [value.(h_sg_putt)[i+1] for i in h_idx_df[!,:idx]]


p_rounds = [pweight_df[!,:weight][i+1]*n_real for i in p_idx_df[!,:idx]]
h_rounds = [hweight_df[!,:weight][i+1]*n_real for i in h_idx_df[!,:idx]]
nodsr_obj_val = DataFrame(obj = [sqrt(objective_value(mod_real))])

nodsr_h_sg_final_df = DataFrame(idx = h_idx_df[!,:idx], course_hole = h_idx_df[!,:course_hole], sg_drive = h_d_label, sg_appr = h_a_label, sg_short = h_s_label, sg_putt = h_p_label,num_rounds = h_rounds)

nodsr_p_final_df = DataFrame(idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label, sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = p_rounds)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_plyr_nodsr.csv", nodsr_p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_hole_nodsr.csv", nodsr_h_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_obj_nodsr.csv", nodsr_obj_val)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4680
Number of nonzeros in inequality constraint Jacobian.:     9360
Number of nonzeros in Lagrangian Hessian.............:   351648

Total number of variables............................:     4680
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

"C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_obj_nodsr.csv"

#  DSR

In [ ]:
## have to run preprocessing to get weights from no dsr first
### WORKING DSR MODEL: 08-24-2020

mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_drive[1:num_plyrs])
@variable(mod_real, h_sg_drive[1:num_holes])
@variable(mod_real, h_dsr_drive[1:num_holes])
@variable(mod_real, p_sg_appr[1:num_plyrs])
@variable(mod_real, h_sg_appr[1:num_holes])
@variable(mod_real, h_dsr_appr[1:num_holes])
@variable(mod_real, p_sg_short[1:num_plyrs])
@variable(mod_real, h_sg_short[1:num_holes])
@variable(mod_real, h_dsr_short[1:num_holes])
@variable(mod_real, p_sg_putt[1:num_plyrs])
@variable(mod_real, h_sg_putt[1:num_holes])
@variable(mod_real, h_dsr_putt[1:num_holes])

p_sg_drive_ = [p_sg_drive[i+1] for i in real_data[!,:p_idx]]
h_sg_drive_ = [h_sg_drive[i+1] for i in real_data[!,:h_idx]]
h_dsr_drive_ = [h_dsr_drive[i+1] for i in real_data[!,:h_idx]]
p_sg_appr_ = [p_sg_appr[i+1] for i in real_data[!,:p_idx]]
h_sg_appr_ = [h_sg_appr[i+1] for i in real_data[!,:h_idx]]
h_dsr_appr_ = [h_dsr_appr[i+1] for i in real_data[!,:h_idx]]
p_sg_short_ = [p_sg_short[i+1] for i in real_data[!,:p_idx]]
h_sg_short_ = [h_sg_short[i+1] for i in real_data[!,:h_idx]]
h_dsr_short_ = [h_dsr_short[i+1] for i in real_data[!,:h_idx]]
p_sg_putt_ = [p_sg_putt[i+1] for i in real_data[!,:p_idx]]
h_sg_putt_ = [h_sg_putt[i+1] for i in real_data[!,:h_idx]]
h_dsr_putt_ = [h_dsr_putt[i+1] for i in real_data[!,:h_idx]]




hsg_wmean_drive = @expression(mod_real, sum(h_sg_drive[i]*(filter(row -> (row.h_idx == i-1) , hweight_df)[!,:weight][1]) for i in 1:num_holes))
hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 


psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 

real_pwvar_drive = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_drive[i]))*(abs(p_sg_drive[i]))) for i in 1:num_plyrs)) # weighted variance
real_pdrive_var_constr = @NLconstraint(mod_real,  real_pwvar_drive == 1.0)

dsr_drive_constr = @constraint(mod_real, [i=1:num_holes], 0 <= h_dsr_drive[i] <= 1) 

sg_drive_apx_constr = @constraint(mod_real, [i=1:num_holes], filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_drive_mean][1] -3 <= h_sg_drive[i] <= filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_drive_mean][1] + 3) 
psg_drive_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_drive_mean][1]-3 <= p_sg_drive[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_drive_mean][1]+3)


hsg_wmean_appr = @expression(mod_real, sum(h_sg_appr[i]*(filter(row -> (row.h_idx == i-1) , hweight_df)[!,:weight][1]) for i in 1:num_holes))
hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 


psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 

real_pwvar_appr = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_appr[i]))*(abs(p_sg_appr[i]))) for i in 1:num_plyrs)) # weighted variance
real_pappr_var_constr = @NLconstraint(mod_real,  real_pwvar_appr == 1.0)

dsr_appr_constr = @constraint(mod_real, [i=1:num_holes], 0 <= h_dsr_appr[i] <= 1)

sg_appr_apx_constr = @constraint(mod_real, [i=1:num_holes], filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_appr_mean][1] -3 <= h_sg_appr[i] <= filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_appr_mean][1] + 3) 
psg_appr_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_appr_mean][1]-3 <= p_sg_appr[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_appr_mean][1] +3)


hsg_wmean_short = @expression(mod_real, sum(h_sg_short[i]*(filter(row -> (row.h_idx == i-1) , hweight_df)[!,:weight][1]) for i in 1:num_holes))
hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 

psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 

real_pwvar_short = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_short[i]))*(abs(p_sg_short[i]))) for i in 1:num_plyrs)) # weighted variance
real_pshort_var_constr = @NLconstraint(mod_real,  real_pwvar_short == 1.0)


dsr_short_constr = @constraint(mod_real, [i=1:num_holes], 0 <= h_dsr_short[i] <= 1) 

sg_short_apx_constr = @constraint(mod_real, [i=1:num_holes], filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_short_mean][1] -3 <= h_sg_short[i] <= filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_short_mean][1] +3) 
psg_short_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_short_mean][1]-3 <= p_sg_short[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_short_mean][1]+3)

hsg_wmean_putt = @expression(mod_real, sum(h_sg_putt[i]*(filter(row -> (row.h_idx == i-1) , hweight_df)[!,:weight][1]) for i in 1:num_holes))
hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0) 

psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0) 

real_pwvar_putt = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_putt[i]))*(abs(p_sg_putt[i]))) for i in 1:num_plyrs)) # weighted variance
real_pputt_var_constr = @NLconstraint(mod_real,  real_pwvar_putt == 1.0)


dsr_putt_constr = @constraint(mod_real, [i=1:num_holes], 0 <= h_dsr_putt[i]<= 1 ) 

sg_putt_apx_constr = @constraint(mod_real, [i=1:num_holes], filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_putt_mean][1] -3 <= h_sg_putt[i] <= filter(row -> (row.h_idx == i-1) , hweight_df)[!,:sg_putt_mean][1] + 3) 
psg_putt_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_putt_mean][1]-3 <= p_sg_putt[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_putt_mean][1]+3)


# p*d - c
@NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i]*h_dsr_drive_[i]*h_dsr_drive_[i] - 2*p_sg_drive_[i]*h_dsr_drive_[i]*h_sg_drive_[i] + h_sg_drive_[i]*h_sg_drive_[i]- 2*p_sg_drive_[i]*h_dsr_drive_[i]*real_data[!,:SG_drive][i] + 2*h_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i]*h_dsr_appr_[i]*h_dsr_appr_[i] - 2*p_sg_appr_[i]*h_dsr_appr_[i]*h_sg_appr_[i] + h_sg_appr_[i]*h_sg_appr_[i]- 2*p_sg_appr_[i]*h_dsr_appr_[i]*real_data[!,:SG_appr][i] + 2*h_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i]*h_dsr_short_[i]*h_dsr_short_[i] - 2*p_sg_short_[i]*h_dsr_short_[i]*h_sg_short_[i] + h_sg_short_[i]*h_sg_short_[i]- 2*p_sg_short_[i]*h_dsr_short_[i]*real_data[!,:SG_short][i] + 2*h_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i]*h_dsr_putt_[i]*h_dsr_putt_[i] - 2*p_sg_putt_[i]*h_dsr_putt_[i]*h_sg_putt_[i] + h_sg_putt_[i]*h_sg_putt_[i]- 2*p_sg_putt_[i]*h_dsr_putt_[i]*real_data[!,:SG_putt][i] + 2*h_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i])  for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

h_idx_df = DataFrame(idx = unique(real_data[!,:h_idx]), course_hole = unique(real_data[!,:course_hole]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_d_label = [value.(p_sg_drive)[i+1] for i in p_idx_df[!,:idx]]
h_d_label = [value.(h_sg_drive)[i+1] for i in h_idx_df[!,:idx]]
hd_d_label = [value.(h_dsr_drive)[i+1] for i in h_idx_df[!,:idx]]

p_a_label = [value.(p_sg_appr)[i+1] for i in p_idx_df[!,:idx]]
h_a_label = [value.(h_sg_appr)[i+1] for i in h_idx_df[!,:idx]]
hd_a_label = [value.(h_dsr_appr)[i+1] for i in h_idx_df[!,:idx]]

p_s_label = [value.(p_sg_short)[i+1] for i in p_idx_df[!,:idx]]
h_s_label = [value.(h_sg_short)[i+1] for i in h_idx_df[!,:idx]]
hd_s_label = [value.(h_dsr_short)[i+1] for i in h_idx_df[!,:idx]]

p_p_label = [value.(p_sg_putt)[i+1] for i in p_idx_df[!,:idx]]
h_p_label = [value.(h_sg_putt)[i+1] for i in h_idx_df[!,:idx]]
hd_p_label = [value.(h_dsr_putt)[i+1] for i in h_idx_df[!,:idx]]

rounds = [filter(row -> (row.p_idx == i) , pweight_df)[!,:weight][1]*n_real for i in p_idx_df[!,:idx]]
obj_val =  objective_value(mod_real)

h_sg_final_df = DataFrame(h_idx = h_idx_df[!,:idx], course_hole = h_idx_df[!,:course_hole], sg_drive = h_d_label, sg_appr = h_a_label, sg_short = h_s_label, sg_putt = h_p_label)
h_dsr_final_df = DataFrame(h_idx = h_idx_df[!,:idx], course_hole = h_idx_df[!,:course_hole], dsr_drive = hd_d_label, dsr_appr = hd_a_label, dsr_short = hd_s_label, dsr_putt = hd_p_label)

p_final_df = DataFrame(p_idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label,sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = rounds)

obj = DataFrame(obj = obj_val)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_byhole_plyr_.csv", p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_byhole_hsg_sg_.csv", h_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_byhole_h_sg_.csv", h_dsr_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_byhole_obj_.csv", obj)